# Multi-level combinatoriality in magpie non-song vocalizations: Call Analysis

The following series of notebooks evaluates variation at the call-level, continuing to follow the tutorial outlined in Sainburg et al., (2020), with some modifications to account for acoustic parameters specific to our study species: the Western Australian magpie (Gymnorhina tibicen dorsalis). Again, further detail on the method, including source code for the analysis used in this paper, can be accessed through the tutorial's repository at [github.com/timsainb/avgn](https://github.com/timsainb/avgn_paper/blob/V2/github.com/timsainb/avgn). 

Data for this study can be accessed [here](https://doi.org/10.26182/s77t-hw04). 

In this notebook I create dataframes for calls and combination data using information in the annotation (Praat textgrid) and audio (WAV) files.

In [1]:
DATASET_ID = "git_repos_call"

In [2]:
import pdb
from joblib import Parallel, delayed
from tqdm.auto import tqdm
import pandas as pd
import librosa
from datetime import datetime
import json
import numpy as np
from pathlib2 import Path

In [3]:
import avgn

In [4]:
from avgn.utils.paths import DATA_DIR, ensure_dir

## Load Data in Original Format

In [5]:
# create a unique datetime identifier for the files output by this notebook 
##(they will be saved in a folder with today's date/time of output creation)
DT_ID = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
DT_ID

'2022-09-12_13-07-28'

In [6]:
#Defining Path to Raw Dataset
RAW_DATASET_LOC = DATA_DIR/"raw"/"magpie"/"individuals"
RAW_DATASET_LOC

WindowsPath('C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-2/data/raw/magpie/individuals')

In [7]:
#grab an array of all textgrid files 
TGLIST = np.array(list((RAW_DATASET_LOC).expanduser().glob('*/*.TextGrid')))
TGLIST = np.array([i for i in TGLIST if i.stem[0] != '.'])
TGLIST[0], len(TGLIST)

(WindowsPath('C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-2/data/raw/magpie/individuals/BWYa_F_MGGY/BWY MGGY Call Combo 1 290719 PM.TextGrid'),
 153)

In [8]:
#array of all WAV files
WAVLIST = np.array(list((RAW_DATASET_LOC).expanduser().glob('*/*.wav')))
WAVLIST[0], len(WAVLIST)

(WindowsPath('C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-2/data/raw/magpie/individuals/BWYa_F_MGGY/BWY MGGY Call Combo 1 290719 PM.wav'),
 153)

## Create dataframes for Calls & Combis


In [9]:
#creating array containing just stem of WAVLIST arrays (i.e. just the file name)
wav_stems = np.array([i.stem for i in WAVLIST])
wav_stems[0]

'BWY MGGY Call Combo 1 290719 PM'

In [10]:
import avgn
from avgn.custom_parsing.magpie_for_repos import (
    get_segments,
    get_calls,
    get_combis
)
from avgn.utils.paths import DATA_DIR

### Calls

In [11]:
#create textrgids again just with unit data
with Parallel(n_jobs=-1, verbose=10) as parallel:
    call_df = parallel(
        delayed(get_calls)(tg, WAVLIST, wav_stems)
        for tg in tqdm(TGLIST)
    )
call_df = pd.concat(call_df)

  0%|          | 0/153 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    4.3s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done  48 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1863s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  61 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0883s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 106 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 126 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 130 out of 153 | elapsed:    4.8s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done 146 out of 153 | elapsed:   

In [12]:
call_df[:6]

,indv,sex,gro,filename,wavloc,tgloc,call_pos_combi,call_start,call_end,call_label
0,MGGY,F,BWYa,BWY MGGY Call Combo 1 290719 PM,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,0,0.753604,0.921160,DSSHDS
1,MGGY,F,BWYa,BWY MGGY Call Combo 1 290719 PM,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,1,0.932017,1.367130,SHSHLH
0,MGGY,F,BWYa,BWY MGGY Call Combo 1 300719 AM,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,0,1.218085,1.308841,DS
1,MGGY,F,BWYa,BWY MGGY Call Combo 1 300719 AM,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,1,1.442686,1.984961,USC
0,MGGY,F,BWYa,BWY MGGY Discrete 1 300719 AM,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,0,1.244022,1.420183,DSSHDS
1,MGGY,F,BWYa,BWY MGGY Discrete 1 300719 AM,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,1,1.471596,1.897371,LH


In [13]:
#Create ID with from 0 to end of range - so that each row has a unique identifier
call_df.insert(0, 'New_ID', range(0, 0 + len(call_df)))

In [16]:
#### Need to save df as pickle bc WAV location paths will no longer work after being imported from the csv (next step)
save_loc = DATA_DIR / DATASET_ID / 'walsh_magpie_call_dataframe.pickle'
ensure_dir(save_loc.as_posix())
call_df.to_pickle(save_loc)

In [15]:
len(call_df)

699

### Combinations

In [18]:
#create textrgids again just with combi data
with Parallel(n_jobs=-1, verbose=10) as parallel:
    combi_df = parallel(
        delayed(get_combis)(tg, WAVLIST, wav_stems)
        for tg in tqdm(TGLIST)
    )
combi_df = pd.concat(combi_df)

  0%|          | 0/153 [00:00<?, ?it/s]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0278s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0660s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  74 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0977s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 130 out of 153 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 146 out of 153 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 153 out of 153 | elapsed:    0.2s finished


In [19]:
combi_df[:6]

,indv,sex,gro,filename,wavloc,tgloc,combi_num,combi_start,combi_end,combi_label
0,MGGY,F,BWYa,BWY MGGY Call Combo 1 290719 P,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,0,0.753604,1.367130,DSSHDS SHSHLH
0,MGGY,F,BWYa,BWY MGGY Call Combo 1 300719 A,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,0,1.218085,1.984961,DS USC
0,MGGY,F,BWYa,BWY MGGY Discrete 1 300719 AM,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,0,1.244022,1.897371,DSSHDS LH
0,MGGY,F,BWYa,BWY MGGY Discrete 4 210519 PM,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,0,0.306330,0.927264,DSSHDS SHLH
0,MGGY,F,BWYa,BWY MGGY Discrete 6 210519 PM,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,0,0.158063,0.503686,DSSHDS SHLH
0,MGGY,F,BWYa,BWY MGGY Excitement 2 170519 P,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,0,0.057590,0.535485,DSSHDS NLSHDS


## Put all data into SQL database

In [20]:
import sqlite3 as sql

In [21]:
con = sql.connect('magpie_call.db')

In [22]:
#turn dataframes into string
call_df = call_df.applymap(str)
combi_df = combi_df.applymap(str)

In [23]:
#connect string dataframes to SQL database
call_df.to_sql('calls', con)
combi_df.to_sql('combis', con)

In [24]:
#first I create dataframe with just ID information - to reduce duplications when combining all data
con = con
c = con.cursor()

c.execute("""
    SELECT DISTINCT New_ID, indv, gro, sex, wavloc, filename
    FROM calls
    """)

IDdf = pd.DataFrame(c.fetchall(), columns = ['New_ID', 'indv', 'gro', 'sex', 'wavloc', 'filename'])
print(IDdf)

    New_ID    indv   gro sex  \
0        0    MGGY  BWYa   F   
1        1    MGGY  BWYa   F   
2        2    MGGY  BWYa   F   
3        3    MGGY  BWYa   F   
4        4    MGGY  BWYa   F   
..     ...     ...   ...  ..   
694    694  MXXGRY    SS   F   
695    695  MXXGRY    SS   F   
696    696  MXXGRY    SS   F   
697    697  MXXGRY    SS   F   
698    698  MXXGRY    SS   F   

                                                wavloc  \
0    C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...   
1    C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...   
2    C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...   
3    C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...   
4    C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...   
..                                                 ...   
694  C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...   
695  C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...   
696  C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...   
697  C:\Users\slwal\anaconda3\envs\

In [25]:
IDdf = pd.DataFrame(IDdf)
IDdf[:8]

,New_ID,indv,gro,sex,wavloc,filename
0,0,MGGY,BWYa,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Call Combo 1 290719 PM
1,1,MGGY,BWYa,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Call Combo 1 290719 PM
2,2,MGGY,BWYa,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Call Combo 1 300719 AM
3,3,MGGY,BWYa,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Call Combo 1 300719 AM
4,4,MGGY,BWYa,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Discrete 1 300719 AM
5,5,MGGY,BWYa,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Discrete 1 300719 AM
6,6,MGGY,BWYa,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Discrete 4 210519 PM
7,7,MGGY,BWYa,F,C:\Users\slwal\anaconda3\envs\PY36\avgn_paper-...,BWY MGGY Discrete 4 210519 PM


In [26]:
#how many rows? >699 means there are duplicates
len(IDdf[:])

699

In [27]:
#dataframe containing start and end times

con = con
c = con.cursor()

c.execute("""
    SELECT DISTINCT New_ID, calls.filename, call_label, call_start, call_end, combi_label, combi_start, combi_end
    FROM calls
    LEFT JOIN combis ON (calls.wavloc == combis.wavloc AND calls.call_start >= combis.combi_start AND calls.call_end <= combis.combi_end)
    """)

df = pd.DataFrame(c.fetchall(), columns = ['New_ID', 'filename', 'call_label', 'call_start', 'call_end', 'combi_label', 'combi_start', 'combi_end'])
df

,New_ID,filename,call_label,call_start,call_end,combi_label,combi_start,combi_end
0,0,BWY MGGY Call Combo 1 290719 PM,DSSHDS,0.7536043330872654,0.9211598748571118,DSSHDS SHSHLH,0.7536043330872654,1.3671299042471126
1,1,BWY MGGY Call Combo 1 290719 PM,SHSHLH,0.932017364606172,1.3671299042471126,DSSHDS SHSHLH,0.7536043330872654,1.3671299042471126
2,2,BWY MGGY Call Combo 1 300719 AM,DS,1.2180851883685253,1.3088406296623487,DS USC,1.2180851883685253,1.9849613064156608
3,3,BWY MGGY Call Combo 1 300719 AM,USC,1.4426860643402732,1.9849613064156608,DS USC,1.2180851883685253,1.9849613064156608
4,4,BWY MGGY Discrete 1 300719 AM,DSSHDS,1.2440220682349485,1.4201833369917516,DSSHDS LH,1.2440220682349485,1.8973712707974528
...,...,...,...,...,...,...,...,...
739,694,SS MXXGRY XXXX JUVENILE Call Combination Serie...,LHDS,0.2397330291871744,0.438800496254343,SHDS LHDS,0.048004454915108796,0.438800496254343
740,695,SS MXXGRY XXXX JUVENILE Call Combination Serie...,SHDS,1.0182589689619694,1.1489829968747414,SHDS LHDS,1.0182589689619694,1.4154765414969186
741,696,SS MXXGRY XXXX JUVENILE Call Combination Serie...,LHDS,1.2187024784282199,1.4154765414969186,SHDS LHDS,1.0182589689619694,1.4154765414969186
742,697,SS MXXGRY XXXX JUVENILE Call Combination Serie...,SHDS,0.11203559171039924,0.2389857420586103,SHDS LHDS,0.11203559171039924,0.512517509303725


In [28]:
#how many rows? >698 means there are duplicates
len(df)

744

In [29]:
from avgn.utils.paths import ensure_dir

## Turn dataframes into csv for processing
Easily allows combining the data from both dataframes and checking for any errors/duplicates that might have occurred during processing. 

In [31]:
#check data directory path
DATA_DIR

WindowsPath('C:/Users/slwal/anaconda3/envs/PY36/avgn_paper-2/data')

In [32]:
save_loc = DATA_DIR / DATASET_ID / 'call_startendtimes_df_to_csv.csv'
ensure_dir(save_loc.as_posix())
df.to_csv(save_loc)

In [33]:
save_loc = DATA_DIR / DATASET_ID / 'call_ID_df_to_csv.csv'
ensure_dir(save_loc)
IDdf.to_csv(save_loc)